In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
136,ABW,North America,Aruba,2020-08-06,171.0,39.0,3.0,0.0,1601.633,365.285,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
346,AFG,Asia,Afghanistan,2020-08-06,36896.0,67.0,1298.0,4.0,947.793,1.721,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
484,AGO,Africa,Angola,2020-08-06,1344.0,180.0,59.0,5.0,40.893,5.477,...,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-05,France,194992.0,0.0,30344.0,0.0,0.0,0.0
0,2020-08-05,China,88302.0,0.0,4684.0,0.0,0.0,0.0
0,2020-08-05,Italy,248702.0,0.0,35204.0,0.0,0.0,0.0
0,2020-08-05,Spain,309873.0,0.0,28530.0,0.0,0.0,0.0
0,2020-08-05,United States,4852519.0,0.0,157520.0,0.0,0.0,0.0
0,2020-08-05,World,18969805.0,0.0,706925.0,0.0,0.0,0.0
0,2020-08-05,United Kingdom,307409.0,0.0,46363.0,0.0,0.0,0.0
0,2020-08-05,Germany,212877.0,0.0,9181.0,0.0,0.0,0.0
0,2020-08-05,Iran,318357.0,0.0,17835.0,0.0,0.0,0.0
0,2020-08-05,Turkey,235852.0,0.0,5782.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-06,France,194992.0,194029.0,30344.0,30305.0,963.0,39.0
0,2020-08-06,China,88302.0,88328.0,4684.0,4677.0,-26.0,7.0
0,2020-08-06,Italy,248702.0,248803.0,35204.0,35181.0,-101.0,23.0
0,2020-08-05,Spain,309873.0,305767.0,28530.0,28499.0,4106.0,31.0
0,2020-08-06,United States,4852519.0,4823891.0,157520.0,158256.0,28628.0,-736.0
0,2020-08-06,World,18969805.0,18793522.0,706925.0,707715.0,176283.0,-790.0
0,2020-08-06,United Kingdom,307409.0,307184.0,46363.0,46364.0,225.0,-1.0
0,2020-08-06,Germany,212877.0,213067.0,9181.0,9175.0,-190.0,6.0
0,2020-08-06,Iran,318357.0,317483.0,17835.0,17802.0,874.0,33.0
0,2020-08-06,Turkey,235852.0,236112.0,5782.0,5784.0,-260.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")